In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display
import re
import requests
from datetime import datetime

# Ticket Melon

In [2]:
chrome_driver_path = '/Users/got/Documents/retune/t+im/chromedriver-mac/chromedriver'
chrome_driver_service = webdriver.chrome.service.Service(chrome_driver_path)

driver = webdriver.Chrome(service=chrome_driver_service)
driver.maximize_window()
driver.get("https://www.ticketmelon.com")

wait = WebDriverWait(driver, 10) #timeout=10 sec

scroll_distance = 1200
scroll_script = f"window.scrollBy(0, {scroll_distance});"
driver.execute_script(scroll_script)

print("scroll to entertainment :" , scroll_distance, "px")

entertainment_button = wait.until(
    EC.element_to_be_clickable(
        (By.CSS_SELECTOR, '.slick-slide.slick-active.slick-current')))
entertainment_button.click()

scroll_distance = 1500
scroll_script = f"window.scrollBy(0, {scroll_distance});"
driver.execute_script(scroll_script)

print("scroll to see more :" , scroll_distance, "px")

see_more_button = driver.find_element(
    By.CSS_SELECTOR, '.HomeStyled__EventSeeMore-sc-125xn4a-2.eMhTTB')
driver.execute_script("arguments[0].click();", see_more_button)

page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

driver.quit()

scroll to entertainment : 1200 px
scroll to see more : 1500 px


In [3]:
elements = soup.find_all('div',
                         class_='MelonEventCardStyled__EventDetail-sc-8kc3ju-2'
                         )  #event-date, event-name, event-location
event_data = []
for element in elements:
    event_info = {
        "title": element.find('p', class_='event-name').text,
        "date": element.find('p', class_='event-date').text,
        "location": element.find('p', class_='event-location').text
    }
    event_data.append(event_info)
melon_concert = pd.DataFrame(event_data)


## modify date

In [4]:
def extract_dates(date_str):
    date_parts = re.findall(r'(\d{1,2}\s*[A-Za-z]{3})', date_str)
    
    if len(date_parts) == 2:
        start_date, end_date = date_parts
    elif len(date_parts) == 1:
        start_date = end_date = date_parts[0]
    else:
        start_date = end_date = None
    
    return start_date.strip(), end_date.strip() 
melon_concert[['startdate', 'enddate']] = melon_concert['date'].apply(extract_dates).apply(pd.Series)

def calculate_year(date_str):
    # Convert month abbreviations to numeric values
    month_values = {
        'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4,
        'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8,
        'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
    }
    
    # Extract the month from the date
    month = date_str.split()[1]
    
    if month_values.get(month):
        if month_values[month] >= 8:
            return 2023
        else:
            return 2024
    else:
        return None  # Handle invalid month

melon_concert['startyear'] = melon_concert['startdate'].apply(calculate_year)
melon_concert['endyear'] = melon_concert['enddate'].apply(calculate_year)
melon_concert['startyear'] = melon_concert['startyear'].astype(str)
melon_concert['endyear'] = melon_concert['endyear'].astype(str)

melon_concert['startdate'] = melon_concert['startdate'] + ' ' + melon_concert['startyear']
melon_concert['startdate'] = pd.to_datetime(melon_concert['startdate'], format='%d %b %Y', errors='coerce')
melon_concert['enddate'] = melon_concert['enddate'] + ' ' + melon_concert['endyear']
melon_concert['enddate'] = pd.to_datetime(melon_concert['enddate'], format='%d %b %Y', errors='coerce')

melon_concert.drop(columns=['startyear', 'endyear'], inplace=True)
melon_concert = melon_concert.sort_values(by=['startdate', 'enddate'])

melon_concert.drop(columns=['date'], inplace=True)

In [5]:
# def extract_dates(date_str):
#     date_parts = re.findall(r'(\d{1,2}\s*[A-Za-z]{3})', date_str)
    
#     if len(date_parts) == 2:
#         start_date, end_date = date_parts
#     elif len(date_parts) == 1:
#         start_date = end_date = date_parts[0]
#     else:
#         start_date = end_date = None
    
#     return start_date.strip(), end_date.strip() 
# melon_concert[['startdate', 'enddate']] = melon_concert['date'].apply(extract_dates).apply(pd.Series)

# display(melon_concert)

In [6]:
# melon_concert.info()

In [7]:
# def calculate_year(date_str):
#     # Convert month abbreviations to numeric values
#     month_values = {
#         'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4,
#         'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8,
#         'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
#     }
    
#     # Extract the month from the date
#     month = date_str.split()[1]
    
#     if month_values.get(month):
#         if month_values[month] >= 6:
#             return 2023
#         else:
#             return 2024
#     else:
#         return None  # Handle invalid month

# melon_concert['year'] = melon_concert['startdate'].apply(calculate_year)

# melon_concert.head(3)

In [8]:
# melon_concert['year'] = melon_concert['year'].astype(str)
# melon_concert['startdate'] = melon_concert['startdate'] + ' ' + melon_concert['year']
# melon_concert['startdate'] = pd.to_datetime(melon_concert['startdate'], format='%d %b %Y', errors='coerce')

# melon_concert['enddate'] = melon_concert['enddate'] + ' ' + melon_concert['year']
# melon_concert['enddate'] = pd.to_datetime(melon_concert['enddate'], format='%d %b %Y', errors='coerce')

In [9]:
melon_concert.head(3)

,title,location,startdate,enddate
5,Monet and Friends Alive Bangkok,"Attraction Hall, 6th Floor, ICONSIAM",2023-09-22,2024-02-14
4,Enchanted Flora Park,Flora Park,2023-11-01,2024-02-29
6,Enchanted Flora Park,Flora Park,2023-11-01,2024-02-29


## liite touch.

In [10]:
# melon_concert.drop(columns=['year'], inplace=True)
# melon_concert = melon_concert.sort_values(by=['startdate', 'enddate'])

In [11]:
# melon_concert.drop(columns=['date'], inplace=True)

In [12]:
display(melon_concert)

,title,location,startdate,enddate
5,Monet and Friends Alive Bangkok,"Attraction Hall, 6th Floor, ICONSIAM",2023-09-22,2024-02-14
4,Enchanted Flora Park,Flora Park,2023-11-01,2024-02-29
6,Enchanted Flora Park,Flora Park,2023-11-01,2024-02-29
7,The Backyard 168: Back to the 90's Campfire Party,"The Backyard 168, Khao Yai",2023-12-01,2024-02-25
8,NuNew 2nd Single Nu Meow Official Goods,Photobook Delivery,2023-12-12,2024-01-12
...,...,...,...,...
126,Splash Pool Party,สวนน้ำ The Mall Korat,2024-04-13,2024-04-16
127,เฟี้ยวติวัล บุรีรัมย์,สนามช้างอารีน่า,2024-04-27,2024-04-27
128,PONG 47 ปี Rock Never Dies,IMPACT ARENA เมืองทองธานี,2024-04-27,2024-04-27
129,เฟี้ยวติวัล ขอนแก่น,ศูนย์ค้าส่งอู้ฟู่,2024-05-11,2024-05-11


In [13]:
melon_concert['vendor'] = 'Ticketmelon'

In [14]:
nan_rows = melon_concert.loc[melon_concert.isna().any(axis=1)]
print(nan_rows)

Empty DataFrame
Columns: [title, location, startdate, enddate, vendor]
Index: []


In [15]:
# oct_con = melon_concert[melon_concert['startdate'].dt.month == 10]
# display(oct_con)

# TTM

## make df

In [16]:
TTMbigdata = requests.get("https://www.thaiticketmajor.com/concert/")

In [36]:
TTMsoup = BeautifulSoup(TTMbigdata.text, "html.parser")
TTMsoup.find_all("div", class_="box-txt")

div_elements = TTMsoup.find_all("div", class_="box-txt")
event_data = []

for div in div_elements:
    event_info = {
        "title": div.find("a", class_="title").text if div.find("a", class_="title") else "",
        "date": div.find("span", class_="datetime").text if div.find("span", class_="datetime") else "",
        "location": div.find("a", class_="venue").text if div.find("a", class_="venue") else ""
    }
    event_data.append(event_info)


ttm_concert = pd.DataFrame(event_data)
ttm_concert = ttm_concert[~ttm_concert['date'].str.contains('Public Sale')]
ttm_concert.head(5)

,title,date,location
0,BRUNO MARS LIVE IN BANGKOK,30 มี.ค. 2567,สนามราชมังคลากีฬาสถาน
1,Last Twilight Final EP. FAN MEETING,26 ม.ค. 2567,สยามภาวลัย ชั้น 6 สยามพารากอน
2,Xdinary Heroes < Break the Brake > World Tour ...,31 มี.ค. 2567,"Ultra Arena, Bravo BKK"
3,อำพลฟูดส์ PRESENTS RS MEETING CONCERT 2024 DAN...,17 ก.พ. 2567,อิมแพ็ค อารีน่า เมืองทองธานี
4,PIT BABE THE LAST RACE,9 ก.พ. 2567,สยามภาวลัย ชั้น 6 สยามพารากอน


## modify date

In [37]:
thai_to_english_month = {
    "ม.ค.": "Jan",
    "ก.พ.": "Feb",
    "มี.ค.": "Mar",
    "เม.ย.": "Apr",
    "พ.ค.": "May",
    "มิ.ย.": "Jun",
    "ก.ค.": "Jul",
    "ส.ค.": "Aug",
    "ก.ย.": "Sep",
    "ต.ค.": "Oct",
    "พ.ย.": "Nov",
    "ธ.ค.": "Dec"
}

def replace_thai_month_with_english(date_str):
    for thai_month, english_month in thai_to_english_month.items():
        date_str = date_str.replace(thai_month, english_month)
    return date_str

# Apply the replacement function to the 'date' column
ttm_concert['date'] = ttm_concert['date'].apply(replace_thai_month_with_english)
ttm_concert.head(1)

,title,date,location
0,BRUNO MARS LIVE IN BANGKOK,30 Mar 2567,สนามราชมังคลากีฬาสถาน


In [38]:
ttm_concert.reset_index(drop=True, inplace=True)
display(ttm_concert)

,title,date,location
0,BRUNO MARS LIVE IN BANGKOK,30 Mar 2567,สนามราชมังคลากีฬาสถาน
1,Last Twilight Final EP. FAN MEETING,26 Jan 2567,สยามภาวลัย ชั้น 6 สยามพารากอน
2,Xdinary Heroes < Break the Brake > World Tour ...,31 Mar 2567,"Ultra Arena, Bravo BKK"
3,อำพลฟูดส์ PRESENTS RS MEETING CONCERT 2024 DAN...,17 Feb 2567,อิมแพ็ค อารีน่า เมืองทองธานี
4,PIT BABE THE LAST RACE,9 Feb 2567,สยามภาวลัย ชั้น 6 สยามพารากอน
5,DISNEY ON ICE PRESENTS 100 YEARS OF WONDER,21 Mar 2567 - 31 Mar 2567,อิมแพ็ค อารีน่า เมืองทองธานี
6,Thailand Phil 2023/24 Season,2 Dec 2566 - 7 Sep 2567,หอประชุมมหิดลสิทธาคาร มหาวิทยาลัยมหิดลศาลายา
7,GMMTV STARLYMPIC 2023,วันเสาร์ที่ 23 ธันวาคม 2566,อิมแพ็ค อารีน่า เมืองทองธานี
8,DMD LAND 2 Wonder Show Concert,13 Jan 2567,อิมแพ็ค อารีน่า เมืองทองธานี
9,MARK TUAN 'The Other Side' ASIA TOUR 2024 in B...,20 Jan 2567 - 21 Jan 2567,อิมแพ็ค อารีน่า เมืองทองธานี


In [39]:
# ttm_concert.loc[1,['date']] = '23 Dec 2566'
rows = [7, 23]
ttm_concert.drop(rows, inplace=True)

In [40]:
# Function to subtract 543 from the year part of a date string
def subtract_543_from_year(date_str):
    # Split the date string by spaces
    parts = date_str.split()
    
    # Check if the date string has the format "DD MMM YYYY - DD MMM YYYY"
    if len(parts) == 7:
        # Extract the year parts
        year_parts = parts[2], parts[-1]
        
        # Subtract 543 from each year part
        updated_year_parts = [str(int(year) - 543) for year in year_parts]
        
        # Replace the year parts in the date string
        parts[2], parts[-1] = updated_year_parts
        
        # Join the updated parts to form the new date string
        updated_date_str = " ".join(parts)
        return updated_date_str
    else:
        # Extract the year part
        year_part = parts[-1]
        
        # Subtract 543 from the year part
        updated_year_part = str(int(year_part) - 543)
        
        # Replace the year part in the date string
        parts[-1] = updated_year_part
        
        # Join the updated parts to form the new date string
        updated_date_str = " ".join(parts)
        return updated_date_str

# Apply the subtraction function to the 'date' column
ttm_concert['date'] = ttm_concert['date'].apply(subtract_543_from_year)

ttm_concert.head(3)


,title,date,location
0,BRUNO MARS LIVE IN BANGKOK,30 Mar 2024,สนามราชมังคลากีฬาสถาน
1,Last Twilight Final EP. FAN MEETING,26 Jan 2024,สยามภาวลัย ชั้น 6 สยามพารากอน
2,Xdinary Heroes < Break the Brake > World Tour ...,31 Mar 2024,"Ultra Arena, Bravo BKK"


In [41]:
display(ttm_concert)

,title,date,location
0,BRUNO MARS LIVE IN BANGKOK,30 Mar 2024,สนามราชมังคลากีฬาสถาน
1,Last Twilight Final EP. FAN MEETING,26 Jan 2024,สยามภาวลัย ชั้น 6 สยามพารากอน
2,Xdinary Heroes < Break the Brake > World Tour ...,31 Mar 2024,"Ultra Arena, Bravo BKK"
3,อำพลฟูดส์ PRESENTS RS MEETING CONCERT 2024 DAN...,17 Feb 2024,อิมแพ็ค อารีน่า เมืองทองธานี
4,PIT BABE THE LAST RACE,9 Feb 2024,สยามภาวลัย ชั้น 6 สยามพารากอน
5,DISNEY ON ICE PRESENTS 100 YEARS OF WONDER,21 Mar 2024 - 31 Mar 2024,อิมแพ็ค อารีน่า เมืองทองธานี
6,Thailand Phil 2023/24 Season,2 Dec 2023 - 7 Sep 2024,หอประชุมมหิดลสิทธาคาร มหาวิทยาลัยมหิดลศาลายา
8,DMD LAND 2 Wonder Show Concert,13 Jan 2024,อิมแพ็ค อารีน่า เมืองทองธานี
9,MARK TUAN 'The Other Side' ASIA TOUR 2024 in B...,20 Jan 2024 - 21 Jan 2024,อิมแพ็ค อารีน่า เมืองทองธานี
10,THE 1st BLUE DRAGON MUSIC AWARDS,20 Jan 2024,สนามราชมังคลากีฬาสถาน


### clean

In [23]:
# ttm_concert = ttm_concert.drop(28)

In [29]:
# display(ttm_concert)

,title,date,location
0,BRUNO MARS LIVE IN BANGKOK,30 Mar 1481,สนามราชมังคลากีฬาสถาน
1,Last Twilight Final EP. FAN MEETING,26 Jan 1481,สยามภาวลัย ชั้น 6 สยามพารากอน
2,Xdinary Heroes < Break the Brake > World Tour ...,31 Mar 1481,"Ultra Arena, Bravo BKK"
3,อำพลฟูดส์ PRESENTS RS MEETING CONCERT 2024 DAN...,17 Feb 1481,อิมแพ็ค อารีน่า เมืองทองธานี
4,PIT BABE THE LAST RACE,9 Feb 1481,สยามภาวลัย ชั้น 6 สยามพารากอน
5,DISNEY ON ICE PRESENTS 100 YEARS OF WONDER,21 Mar 1481 - 31 Mar 1481,อิมแพ็ค อารีน่า เมืองทองธานี
6,Thailand Phil 2023/24 Season,2 Dec 1480 - 7 Sep 1481,หอประชุมมหิดลสิทธาคาร มหาวิทยาลัยมหิดลศาลายา
8,DMD LAND 2 Wonder Show Concert,13 Jan 1481,อิมแพ็ค อารีน่า เมืองทองธานี
9,MARK TUAN 'The Other Side' ASIA TOUR 2024 in B...,20 Jan 1481 - 21 Jan 1481,อิมแพ็ค อารีน่า เมืองทองธานี
10,THE 1st BLUE DRAGON MUSIC AWARDS,20 Jan 1481,สนามราชมังคลากีฬาสถาน


In [42]:
def extract_dates(date_str):
    date_parts = re.findall(r'(\d{1,2}\s*[A-Za-z]{3}\s*\d{4})', date_str)
    
    if len(date_parts) == 2:
        start_date, end_date = date_parts
    elif len(date_parts) == 1:
        start_date = end_date = date_parts[0]
    else:
        start_date = end_date = None
    
    return start_date.strip(), end_date.strip()

ttm_concert[['startdate', 'enddate']] =ttm_concert['date'].apply(extract_dates).apply(pd.Series)
ttm_concert.head(3)

,title,date,location,startdate,enddate
0,BRUNO MARS LIVE IN BANGKOK,30 Mar 2024,สนามราชมังคลากีฬาสถาน,30 Mar 2024,30 Mar 2024
1,Last Twilight Final EP. FAN MEETING,26 Jan 2024,สยามภาวลัย ชั้น 6 สยามพารากอน,26 Jan 2024,26 Jan 2024
2,Xdinary Heroes < Break the Brake > World Tour ...,31 Mar 2024,"Ultra Arena, Bravo BKK",31 Mar 2024,31 Mar 2024


In [43]:
def convert_date_format(date_str):
    # Parse the date string into a datetime object
    date_obj = datetime.strptime(date_str, '%d %b %Y')
    
    # Format the datetime object as 'YYYY-MM-DD'
    formatted_date = date_obj.strftime('%Y-%m-%d')
    
    return formatted_date

ttm_concert['startdate'] = ttm_concert['startdate'].apply(convert_date_format)
ttm_concert['enddate'] = ttm_concert['enddate'].apply(convert_date_format)

ttm_concert.head()

,title,date,location,startdate,enddate
0,BRUNO MARS LIVE IN BANGKOK,30 Mar 2024,สนามราชมังคลากีฬาสถาน,2024-03-30,2024-03-30
1,Last Twilight Final EP. FAN MEETING,26 Jan 2024,สยามภาวลัย ชั้น 6 สยามพารากอน,2024-01-26,2024-01-26
2,Xdinary Heroes < Break the Brake > World Tour ...,31 Mar 2024,"Ultra Arena, Bravo BKK",2024-03-31,2024-03-31
3,อำพลฟูดส์ PRESENTS RS MEETING CONCERT 2024 DAN...,17 Feb 2024,อิมแพ็ค อารีน่า เมืองทองธานี,2024-02-17,2024-02-17
4,PIT BABE THE LAST RACE,9 Feb 2024,สยามภาวลัย ชั้น 6 สยามพารากอน,2024-02-09,2024-02-09


In [32]:
# ttm_concert.astype(da)

In [44]:
ttm_concert['startdate'] = pd.to_datetime(ttm_concert['startdate'])
ttm_concert['enddate'] = pd.to_datetime(ttm_concert['enddate'])
ttm_concert = ttm_concert.sort_values(['startdate', 'enddate'])
ttm_concert = ttm_concert.drop(columns=['date'], axis=1)
display(ttm_concert)

,title,location,startdate,enddate
6,Thailand Phil 2023/24 Season,หอประชุมมหิดลสิทธาคาร มหาวิทยาลัยมหิดลศาลายา,2023-12-02,2024-09-07
8,DMD LAND 2 Wonder Show Concert,อิมแพ็ค อารีน่า เมืองทองธานี,2024-01-13,2024-01-13
10,THE 1st BLUE DRAGON MUSIC AWARDS,สนามราชมังคลากีฬาสถาน,2024-01-20,2024-01-20
9,MARK TUAN 'The Other Side' ASIA TOUR 2024 in B...,อิมแพ็ค อารีน่า เมืองทองธานี,2024-01-20,2024-01-21
11,RBSO 2024 : Music of Anime,ศูนย์วัฒนธรรมแห่งประเทศไทย หอประชุมใหญ่,2024-01-25,2024-01-26
1,Last Twilight Final EP. FAN MEETING,สยามภาวลัย ชั้น 6 สยามพารากอน,2024-01-26,2024-01-26
12,IVE THE 1ST WORLD TOUR < SHOW WHAT I HAVE > IN...,อิมแพ็ค อารีน่า เมืองทองธานี,2024-01-27,2024-01-27
13,คอนเสิร์ต ''King of Rock N' Roll Elvis Presley'',ศาลาเฉลิมกรุง,2024-01-28,2024-01-28
14,THE GOLDEN MELODY CONCERT,ศูนย์วัฒนธรรมแห่งประเทศไทย หอประชุมใหญ่,2024-02-03,2024-02-03
16,"JAM FANCON THE NEW JOURNEY""ขอแจมอีกที""",เมืองไทยรัชดาลัย เธียเตอร์,2024-02-03,2024-02-03


In [45]:
ttm_concert['vendor'] = 'TTM'

In [46]:
display(ttm_concert)

,title,location,startdate,enddate,vendor
6,Thailand Phil 2023/24 Season,หอประชุมมหิดลสิทธาคาร มหาวิทยาลัยมหิดลศาลายา,2023-12-02,2024-09-07,TTM
8,DMD LAND 2 Wonder Show Concert,อิมแพ็ค อารีน่า เมืองทองธานี,2024-01-13,2024-01-13,TTM
10,THE 1st BLUE DRAGON MUSIC AWARDS,สนามราชมังคลากีฬาสถาน,2024-01-20,2024-01-20,TTM
9,MARK TUAN 'The Other Side' ASIA TOUR 2024 in B...,อิมแพ็ค อารีน่า เมืองทองธานี,2024-01-20,2024-01-21,TTM
11,RBSO 2024 : Music of Anime,ศูนย์วัฒนธรรมแห่งประเทศไทย หอประชุมใหญ่,2024-01-25,2024-01-26,TTM
1,Last Twilight Final EP. FAN MEETING,สยามภาวลัย ชั้น 6 สยามพารากอน,2024-01-26,2024-01-26,TTM
12,IVE THE 1ST WORLD TOUR < SHOW WHAT I HAVE > IN...,อิมแพ็ค อารีน่า เมืองทองธานี,2024-01-27,2024-01-27,TTM
13,คอนเสิร์ต ''King of Rock N' Roll Elvis Presley'',ศาลาเฉลิมกรุง,2024-01-28,2024-01-28,TTM
14,THE GOLDEN MELODY CONCERT,ศูนย์วัฒนธรรมแห่งประเทศไทย หอประชุมใหญ่,2024-02-03,2024-02-03,TTM
16,"JAM FANCON THE NEW JOURNEY""ขอแจมอีกที""",เมืองไทยรัชดาลัย เธียเตอร์,2024-02-03,2024-02-03,TTM


# The Concert

In [47]:
chrome_driver_path = '/Users/got/Documents/retune/t+im/chromedriver-mac/chromedriver'
chrome_driver_service = webdriver.chrome.service.Service(chrome_driver_path)
driver = webdriver.Chrome(service=chrome_driver_service)
driver.get("https://www.theconcert.com/concert")

wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))

last_height = 0
print(last_height)
time.sleep(5)
while(True) : 
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(4)
    new_height = driver.execute_script("return document.body.scrollHeight")
    print(new_height)
    if new_height == last_height :
        break 
    else :
        last_height = new_height

wait = WebDriverWait(driver, 10)
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

0
1496
1656
2619
3922
4592
4592


In [48]:
elements = soup.find_all('div',
                         class_='recom-detail-box'
                         )  #conert-recom-title, concert-recom-date, concert-recom-date, concert-recom-local
event_data = []
for element in elements:
    event_info = {
        "title": element.find('div', class_='concert-recom-title').text,
        "date": element.find('div', class_='concert-recom-date').text,
        "location": element.find('div', class_='concert-recom-local').text
    }
    event_data.append(event_info)

the_concert = pd.DataFrame(event_data)
the_concert.head(3)

,title,date,location
0,เขาตาง๊อก เฟสติวัล 2024,12 - 14 ม.ค. 24,ทัมทิมสยาม05
1,The Concert Nightlife presents 1Mill x FIIXD x...,12 - 13 ม.ค. 24,White Noise
2,หนัดเพื่อชีWeed,13 ม.ค. 24,ChangChui


## mo date

In [49]:
thai_to_english_month = {
    "ม.ค.": "Jan",
    "ก.พ.": "Feb",
    "มี.ค.": "Mar",
    "เม.ย.": "Apr",
    "พ.ค.": "May",
    "มิ.ย.": "Jun",
    "ก.ค.": "Jul",
    "ส.ค.": "Aug",
    "ก.ย.": "Sep",
    "ต.ค.": "Oct",
    "พ.ย.": "Nov",
    "ธ.ค.": "Dec"
}

def replace_thai_month_with_english(date_str):
    for thai_month, english_month in thai_to_english_month.items():
        date_str = date_str.replace(thai_month, english_month)
    return date_str

the_concert['date'] = the_concert['date'].apply(replace_thai_month_with_english)
display(the_concert)


,title,date,location
0,เขาตาง๊อก เฟสติวัล 2024,12 - 14 Jan 24,ทัมทิมสยาม05
1,The Concert Nightlife presents 1Mill x FIIXD x...,12 - 13 Jan 24,White Noise
2,หนัดเพื่อชีWeed,13 Jan 24,ChangChui
3,The Concert Present ลาบกรุงคอนสด x JINTA,13 Jan 24,ลาบกรุง
4,Emo Pop Punk,14 - 15 Jan 24,DECOMMUNE
5,Urban Music Festival,20 Jan 24,fortune tower
6,ON GREEN MUSIC FESTIVAL,20 Jan 24,ESC PARK RANGSIT
7,คลั่งรัก,20 Jan 24,Union Mall Hall 1
8,GasolineSrinakarin PRESENT ไททศมิตร,23 Jan 24,Gasoline Bar - Srinakarin
9,“barenbliss PRESENTS GO WENT GAME WITH COPTER ...,27 Jan 24,LIDO Connect Hall 2


In [50]:
def change_year(date_str):
    # Use regular expression to find and replace the year
    updated_date_str = re.sub(r'\b23$', '2023', date_str)
    updated_date_str = re.sub(r'\b24$', '2024', updated_date_str)
    return updated_date_str

the_concert['date'] = the_concert['date'].apply(change_year)
the_concert.head(1)

,title,date,location
0,เขาตาง๊อก เฟสติวัล 2024,12 - 14 Jan 2024,ทัมทิมสยาม05


In [51]:
def split_date(date_str):
    parts = date_str.split('-')
    start_date = parts[0].strip()
    
    if len(parts) > 1:
        end_date = parts[1].strip()
        end_day, end_month, year = end_date.split()
        try:
            start_day, start_month = start_date.split()
            print(start_day, start_month)

        except ValueError:
            start_day = start_date
            start_month = end_month  # or any other suitable default value

        start_date = f"{start_day} {start_month} {year}"
    else:
        end_date = start_date
        
    return start_date, end_date
# Apply the function to the 'date' column
the_concert[['startdate', 'enddate']] = the_concert['date'].apply(split_date).apply(pd.Series)
the_concert.head(3)

,title,date,location,startdate,enddate
0,เขาตาง๊อก เฟสติวัล 2024,12 - 14 Jan 2024,ทัมทิมสยาม05,12 Jan 2024,14 Jan 2024
1,The Concert Nightlife presents 1Mill x FIIXD x...,12 - 13 Jan 2024,White Noise,12 Jan 2024,13 Jan 2024
2,หนัดเพื่อชีWeed,13 Jan 2024,ChangChui,13 Jan 2024,13 Jan 2024


## little touch

In [52]:
the_concert = the_concert.drop(columns=['date'], axis=1)
the_concert['startdate'] = pd.to_datetime(the_concert['startdate'], format='%d %b %Y', errors='coerce')
the_concert['enddate'] = pd.to_datetime(the_concert['enddate'], format='%d %b %Y', errors='coerce')

the_concert = the_concert.sort_values(['startdate', 'enddate'])
the_concert.reset_index(drop=True, inplace=True)
display(the_concert)

,title,location,startdate,enddate
0,The Concert Nightlife presents 1Mill x FIIXD x...,White Noise,2024-01-12,2024-01-13
1,เขาตาง๊อก เฟสติวัล 2024,ทัมทิมสยาม05,2024-01-12,2024-01-14
2,หนัดเพื่อชีWeed,ChangChui,2024-01-13,2024-01-13
3,The Concert Present ลาบกรุงคอนสด x JINTA,ลาบกรุง,2024-01-13,2024-01-13
4,Emo Pop Punk,DECOMMUNE,2024-01-14,2024-01-15
5,ONLY MONDAY live at Come Again Cafe & Restaurant,Come Again Cafe & Restaurant,2024-01-17,2024-01-17
6,Urban Music Festival,fortune tower,2024-01-20,2024-01-20
7,ON GREEN MUSIC FESTIVAL,ESC PARK RANGSIT,2024-01-20,2024-01-20
8,คลั่งรัก,Union Mall Hall 1,2024-01-20,2024-01-20
9,GasolineSrinakarin PRESENT ไททศมิตร,Gasoline Bar - Srinakarin,2024-01-23,2024-01-23


In [53]:
the_concert['vendor'] = 'The Concert'

In [54]:
nan_rows = the_concert.loc[the_concert.isna().any(axis=1)]
print(nan_rows)

Empty DataFrame
Columns: [title, location, startdate, enddate, vendor]
Index: []


# Combined 

In [55]:
# df = pd.concat([melon_concert, ttm_concert, the_concert])
df = pd.concat([melon_concert, the_concert])
df = df.sort_values(['startdate', 'enddate'])
df.reset_index(drop=True, inplace=True)
display(df)

,title,location,startdate,enddate,vendor
0,Monet and Friends Alive Bangkok,"Attraction Hall, 6th Floor, ICONSIAM",2023-09-22,2024-02-14,Ticketmelon
1,Enchanted Flora Park,Flora Park,2023-11-01,2024-02-29,Ticketmelon
2,Enchanted Flora Park,Flora Park,2023-11-01,2024-02-29,Ticketmelon
3,The Backyard 168: Back to the 90's Campfire Party,"The Backyard 168, Khao Yai",2023-12-01,2024-02-25,Ticketmelon
4,NuNew 2nd Single Nu Meow Official Goods,Photobook Delivery,2023-12-12,2024-01-12,Ticketmelon
...,...,...,...,...,...
163,หนัดเพื่อชีWeed 4 พค. - ไววิทย์,ChangChui,2024-05-04,2024-05-04,The Concert
164,เฟี้ยวติวัล ขอนแก่น,ศูนย์ค้าส่งอู้ฟู่,2024-05-11,2024-05-11,Ticketmelon
165,เฟี้ยวติวัล กรุงเทพฯ,ลานตะวันนา 2,2024-05-19,2024-05-19,Ticketmelon
166,หนัดเพื่อชีWeed 1 มิ.ย. - Greasy Cafe,ChangChui,2024-06-01,2024-06-01,The Concert


In [56]:
path = '/Users/got/Documents/retune/concert_data/concert_data.csv'
df.to_csv(path, index=False)